## Домашнее задание к лекции "Статистика. Практика"
### Задание 1
Вернемся к набору данных о видеоиграх.

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?

2) Критикам нравятся больше игры на PC или на PS4?

3) Критикам больше нравятся стрелялки или стратегии?

Для каждого вопроса:

сформулируйте нулевую и альтернативную гипотезы;
выберите пороговый уровень статистической значимости;
опишите полученные результаты статистического теста.

In [354]:
import pandas as pd
import numpy as np
import re
from scipy import stats as st

In [355]:
df = pd.read_csv('video_games_sales.csv')

### 1) Как критики относятся к спортивным играм?
Гипотеза Н0: Критики положительно оценивают спортивные игры (оценка >=7)

Гипотеза H1: Оценка критиков менее 7 баллов

In [356]:
df_sport = df[df.Genre == 'Sports']
df_sports = df_sport[['Rank', 'Name', 'basename','Critic_Score']].dropna()
alpha = 0.05
result = st.ttest_1samp(df_sports['Critic_Score'],7)
print(result)
if (result.statistic < 0) & (result.pvalue / 2 < alpha):
    print('Отвергаем нулевую гипотезу, среднее меньше 7')
else:
    print('Не отвергаем нулевую гипотезу, спортивные игры положительно оцениваются критиками.')

Ttest_1sampResult(statistic=5.197156914436623, pvalue=2.671410902970997e-07)
Не отвергаем нулевую гипотезу, спортивные игры положительно оцениваются критиками.


#### Вывод:

С 95% уровнем достоверности мы можем утверждать что игры жанра спорт большинство критиков оценивает или положительно или удовлетворительно.

#### 2) Критикам нравятся больше игры на PC или на PS4?
Гипотеза Н0: Критики одинаково удовлетворены играми на PC и на PS4.

Гипотеза H1: Критики по-разному относятся к платформам.

In [357]:
df_P = df[df.Platform == 'PC']
df_PC = df_P[['Rank', 'Name', 'basename','Critic_Score']].dropna()
df_PS = df[df.Platform == 'PS4']
df_PS4 = df_PS[['Rank', 'Name', 'basename','Critic_Score']].dropna()
result = st.ttest_ind(df_PC['Critic_Score'], df_PS4['Critic_Score'], equal_var=False)
print(result)
if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики по-разному относятся к платформам.')
else:
    print('Не отвергаем нулевую гипотезу')


Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем нулевую гипотезу, критики по-разному относятся к платформам.


In [358]:
# Проверяем как критики относятся к играм на PC и PS 4
# Гипотеза Н0: Критики положительно оценивают игры на PC/PS4 (оценка >= из списка points)
#Гипотеза H1: Оценка критиков менее "point" баллов
points = [10,9,8,7,6,5,4,3,2,1,0]
list_PC = []
list_PS4 = []
hyp_PC = ''
hyp_PS = ''
for point in points:
    result_PC = st.ttest_1samp(df_PC['Critic_Score'],point)
    if (result_PC.statistic < 0) & (result_PC.pvalue / 2 < alpha):
        hyp_PC = 'H1'
    else:
        hyp_PC = 'H0'
    result_PS = st.ttest_1samp(df_PS4['Critic_Score'],point)
    if (result_PS.statistic < 0) & (result_PS.pvalue / 2 < alpha):
        hyp_PS = 'H1'
    else:
        hyp_PS = 'H0'
    if hyp_PC != hyp_PS:
        print('балл:',point)
        print('Гипотеза PC:', hyp_PC)
        print('Гипотеза PS4:', hyp_PS)
        if hyp_PC =='H1':
            print('С 95% достоверностью можем утверждать что оценки критиков выше у игр на PS4.')
        else:
            print('С 95% достоверностью можем утверждать что оценки критиков выше у игр на PC.')
        


балл: 8
Гипотеза PC: H1
Гипотеза PS4: H0
С 95% достоверностью можем утверждать что оценки критиков выше у игр на PS4.


#### 3) Критикам больше нравятся стрелялки или стратегии?
Гипотеза Н0: Критики относятся одинаково как к стрелялкам, так и к стратегиям.

Гипотеза Н1: Критики относятся по-разному.

In [359]:
alpha2 = 0.05
df_shoot = df[df.Genre == 'Shooter']
df_shooters = df_shoot[['Rank', 'Name', 'basename','Critic_Score']].dropna()
df_strat = df[df.Genre == 'Strategy']
df_strategies = df_strat[['Rank', 'Name', 'basename','Critic_Score']].dropna()
result = st.ttest_ind(df_shooters['Critic_Score'], df_strategies['Critic_Score'], equal_var=False)
print(result)
if (result.pvalue < alpha2):
    print('Отвергаем нулевую гипотезу, критики по-разному относятся к жанрам.')
else:
    print('Не отвергаем нулевую гипотезу')


Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Не отвергаем нулевую гипотезу


### Задание 2
Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

1) Приведите весь текст к нижнему регистру;

2) Удалите мусорные символы;

3) Удалите стоп-слова;

4) Приведите все слова к нормальной форме;

5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(df.Message)

names = tfidf.get_feature_names()

tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=cname)

Можете поэкспериментировать с параметрами TfidfVectorizer;

6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;

7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;

8) Опишите результаты при помощи confusion_matrix;

9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [360]:
df_spam = pd.read_csv('spam.csv')
df_spam

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [361]:
df_message_low = df_spam['Message'].str.lower()
df_category_low = df_spam['Category'].str.lower()

In [362]:
def only_word(row):
    return re.sub('[\W_]+',' ', row)

In [363]:
df_message_word = df_message_low.apply(only_word)
df_category_word = df_category_low.apply(only_word)
df_message_word

0       go until jurong point crazy available only in ...
1                                ok lar joking wif u oni 
2       free entry in 2 a wkly comp to win fa cup fina...
3            u dun say so early hor u c already then say 
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the 2nd time we have tried 2 contact u...
5568                 will ü b going to esplanade fr home 
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: Message, Length: 5572, dtype: object

In [364]:
import nltk
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))
def without_sw(row):
    list_row = row.split(' ')
    for el in list_row:
        if el in stopwords_set:
            del el
    return ' '.join(list_row)

In [365]:
sw = df_message_word.apply(without_sw)
sw

0       go until jurong point crazy available only in ...
1                                ok lar joking wif u oni 
2       free entry in 2 a wkly comp to win fa cup fina...
3            u dun say so early hor u c already then say 
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the 2nd time we have tried 2 contact u...
5568                 will ü b going to esplanade fr home 
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: Message, Length: 5572, dtype: object

In [366]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(row):
    list_row1 = row.split(' ')
    list_lem =[]
    for el in list_row1:
        list_lem.append(wordnet_lemmatizer.lemmatize(el))
    return ' '.join(list_lem)

In [367]:
lem = sw.apply(lemmatize)
lem

0       go until jurong point crazy available only in ...
1                                ok lar joking wif u oni 
2       free entry in 2 a wkly comp to win fa cup fina...
3            u dun say so early hor u c already then say 
4       nah i don t think he go to usf he life around ...
                              ...                        
5567    this is the 2nd time we have tried 2 contact u...
5568                 will ü b going to esplanade fr home 
5569    pity wa in mood for that so any other suggestion 
5570    the guy did some bitching but i acted like i d...
5571                              rofl it true to it name
Name: Message, Length: 5572, dtype: object

In [368]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(lem)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray())

In [369]:
tfidf_matrix

,0,1,2,3,4,5,6,7,8,9,...,8159,8160,8161,8162,8163,8164,8165,8166,8167,8168
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [370]:
df_category_word.replace(['ham', 'spam'], [1, 0], inplace=True) 

In [371]:
df_category_word

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: int64

In [372]:
tfidf_matrix['cat'] = df_category_word

In [373]:
tfidf_matrix

,0,1,2,3,4,5,6,7,8,9,...,8160,8161,8162,8163,8164,8165,8166,8167,8168,cat
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [374]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(tfidf_matrix, test_size=0.3, random_state=42)

In [375]:
y_train, y_test = data_train['cat'], data_test['cat']
del data_train['cat']
del data_test['cat']

In [376]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()

In [377]:
logreg.fit(data_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [378]:
logreg.predict(data_test)

array([1, 1, 1, ..., 1, 0, 1], dtype=int64)

In [379]:
result = pd.DataFrame([y_test.values, logreg.predict(data_test)]).T
print(result)

      0  1
0     1  1
1     1  1
2     1  1
3     1  1
4     1  1
...  .. ..
1667  1  1
1668  0  0
1669  1  1
1670  0  0
1671  1  1

[1672 rows x 2 columns]


In [380]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, logreg.predict(data_test))

0.9623205741626795

In [381]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, logreg.predict(data_test))

array([[ 165,   59],
       [   4, 1444]], dtype=int64)

### Выводы
Модель показала неплохие результаты, в частности, точность прогнозирования составила 96,23%.
Из отрицательных результатов следует выделить то, что модель показала большее число ложно-положительных решений более 3,5%. Т.е. модель определила в не-спам 59 спам-сообщений.

In [382]:
y_val = tfidf_matrix['cat']
del tfidf_matrix['cat']
tfidf_matrix

,0,1,2,3,4,5,6,7,8,9,...,8159,8160,8161,8162,8163,8164,8165,8166,8167,8168
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [383]:
final_pred = pd.DataFrame([y_val,logreg.predict(tfidf_matrix)]).T
final_pred['sum'] = final_pred['cat'] + final_pred['Unnamed 0']
del final_pred['cat']
final_pred.rename(columns = {'Unnamed 0':'Category_predict'}, inplace=True)
final_pred

,Category_predict,sum
0,1,2
1,1,2
2,0,0
3,1,2
4,1,2
...,...,...
5567,0,0
5568,1,2
5569,1,2
5570,1,2


In [384]:
final_pred.reset_index()
df_spam.reset_index()

,index,Category,Message
0,0,ham,"Go until jurong point, crazy.. Available only ..."
1,1,ham,Ok lar... Joking wif u oni...
2,2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,3,ham,U dun say so early hor... U c already then say...
4,4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...,...
5567,5567,spam,This is the 2nd time we have tried 2 contact u...
5568,5568,ham,Will ü b going to esplanade fr home?
5569,5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,5570,ham,The guy did some bitching but I acted like i'd...


In [385]:
df_spam_fin = df_spam.join(final_pred, how = 'left')
df_spam_fin

,Category,Message,Category_predict,sum
0,ham,"Go until jurong point, crazy.. Available only ...",1,2
1,ham,Ok lar... Joking wif u oni...,1,2
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,0
3,ham,U dun say so early hor... U c already then say...,1,2
4,ham,"Nah I don't think he goes to usf, he lives aro...",1,2
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,0,0
5568,ham,Will ü b going to esplanade fr home?,1,2
5569,ham,"Pity, * was in mood for that. So...any other s...",1,2
5570,ham,The guy did some bitching but I acted like i'd...,1,2


In [386]:
df_spam_fin.rename(columns={'Category': 'Category_fact'}, inplace=True)
df_spam_fin.Category_predict.replace([1, 0], ['ham', 'spam'], inplace=True)

In [387]:
df_spam_final = df_spam_fin[(df_spam_fin['sum']==1)]
del df_spam_final['sum']
df_spam_final[['Category_fact', 'Category_predict', 'Message']]

,Category_fact,Category_predict,Message
5,spam,ham,FreeMsg Hey there darling it's been 3 week's n...
15,spam,ham,"XXXMobileMovieClub: To use your credit, click ..."
19,spam,ham,England v Macedonia - dont miss the goals/team...
45,ham,spam,No calls..messages..missed calls
54,spam,ham,SMS. ac Sptv: The New Jersey Devils and the De...
...,...,...,...
5427,spam,ham,Santa Calling! Would your little ones like a c...
5449,spam,ham,"Latest News! Police station toilet stolen, cop..."
5466,spam,ham,http//tms. widelive.com/index. wml?id=820554ad...
5537,spam,ham,Want explicit SEX in 30 secs? Ring 02073162414...
